# model 1 把所有normalize換成 batch normalize, 所有nonlinear 換成leaky relu, 在fpn最前面加入norm並且在所有相加的部分前加入norm


In [ ]:
import torch
import cv2
import os
import numpy as np
import shutil
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from PIL import Image
import torch.nn as nn
import yaml
import random
from google.colab import files
import sys
import time
from torch.utils.data import random_split
device = "cuda" if torch.cuda.is_available() else "cpu" # 檢查是否有可用的 CUDA 設備（通常是顯卡，支援 GPU 運算），如果有，就將 device 變數設置為 "cuda"，否則設置為 "cpu"。
# from Model.functions.functions import *
# from Model.Data_Process.data_processing import *

In [ ]:
class Upsample(nn.Module): # this
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(in_channels,  # this conv let the size unchanged
                                        in_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)

    def forward(self, x):
        x = torch.nn.functional.interpolate(
            x, scale_factor=2.0, mode="nearest") # double the size
        if self.with_conv:
            x = self.conv(x)
        return x


In [ ]:
class Downsample(nn.Module):
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,  # halves the size
                                        in_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0, 1, 0, 1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0) # 此動作相當於在每個圖片的channel的右邊下面pad 0
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, *, in_channels, out_channels=None, conv_shortcut=False,
                 dropout, temb_channels=512):
        super().__init__()
        self.temb_channels = temb_channels
        self.in_channels = in_channels
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        out_channels = in_channels if out_channels is None else out_channels
        self.out_channels = out_channels
        self.use_conv_shortcut = conv_shortcut

        self.norm1 =nn.BatchNorm2d(in_channels)     # 這裡上面define的Normalize有點像是class的感覺
        self.conv1 = torch.nn.Conv2d(in_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        self.temb_proj = torch.nn.Linear(temb_channels,
                                         out_channels)
        self.norm2 = nn.BatchNorm2d(out_channels)
        self.dropout = torch.nn.Dropout(dropout) # param為機率
        self.conv2 = torch.nn.Conv2d(out_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        if self.in_channels != self.out_channels:
            if self.use_conv_shortcut:
                self.conv_shortcut = torch.nn.Conv2d(in_channels,   # size unchanged
                                                     out_channels,
                                                     kernel_size=3,
                                                     stride=1,
                                                     padding=1)
            else:
                self.nin_shortcut = torch.nn.Conv2d(in_channels,    # size unchanged
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0)


    def forward(self, x, temb):
        h = x
        h = self.norm1(h)    # normalize

        h = self.Lrelu(h)  # sigmoid
        h = self.conv1(h)    # channel become out_channel

        h = h + self.temb_proj(self.Lrelu(temb))[:, :, None, None] # 後面加入None增加空的維度，針對temb_proj(nonlinearity(temb))使用，使其可以跟h相加，但是是使用broadcasting的方式
        h = self.norm2(h)
        h = self.Lrelu(h)
        h = self.dropout(h)
        h = self.conv2(h)

        if self.in_channels != self.out_channels:  # 如果inchannel和outchannel不同需要把輸入值channel也調整成一樣，用上conv2D，若inchannel和outchannel一樣就直接加
            if self.use_conv_shortcut:
                x = self.conv_shortcut(x)
            else:
                x = self.nin_shortcut(x)

        return x+h

In [ ]:
class AttnBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = nn.BatchNorm2d(in_channels)
        self.q = torch.nn.Conv2d(in_channels, # in_cha == out_cha and the kernel size = 1, and size unchanged
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b, c, h, w = q.shape # (batch, channel, height, width)
        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)   # b,hw,c.  # 此兩變換(reshape + permute)詳細情形如下格，簡單來說最外圈還是每一個data(一張照片)，
                                # 往內一圈則是把該資料的所有channel(rgb)的同個位置放在一起
        k = k.reshape(b, c, h*w)  # b,c,hw # 單一此變換則是外圈是data，向內一圈則是該data一個channel內所有的值
        w_ = torch.bmm(q, k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        # 注意這邊是q * k，是把不同channels的同個位置變成vector然後內積，這樣跟cnn最大的差別是cnn只會在同一個區塊做相關性，attention卻在channels中的每個位置會相互做相關性
        w_ = w_ * (int(c)**(-0.5)) # 為何不是 * (int(h * w) ** (-0.5))?
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = v.reshape(b, c, h*w)
        w_ = w_.permute(0, 2, 1)   # b,hw,hw (first hw of k, second of q)
        # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = torch.bmm(v, w_)
        h_ = h_.reshape(b, c, h, w)

        h_ = self.proj_out(h_)

        return x+h_

In [ ]:
class DownsampleFPN(nn.Module):
    def __init__(self, in_channels, out_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,  # halves the size
                                        out_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0, 1, 0, 1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0) # 此動作相當於在每個圖片的channel的右邊下面pad 0
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x

In [ ]:
class UpsampleFPN(nn.Module):
    def __init__(self, in_channels, out_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(int(in_channels),  # this conv let the size unchanged
                                        int(out_channels),
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)
    def forward(self, x):
        x = torch.nn.functional.interpolate(
            x, scale_factor=2.0, mode="nearest") # double the size
        if self.with_conv:
            x = self.conv(x)

        return x

In [ ]:
class FPN(nn.Module):  # 此處預設每次的resolutions都是上一次的一半 第一次的resolution是原圖的 1/4
    def __init__(self, config):
        super().__init__()
        resolutions = config['model']['FPN_conv_res'].copy()

        # resolutions = [64, 128, 256, 512]
        self.resolutions = resolutions.copy()

        # self.target_channel = int(resolutions[0] / 2)
        self.target_channel = config['model']['FPN_target_C']

        resolutions.insert(0, 3)
        # self.resolutions = resolutions # which is list
        self.ConvList = nn.ModuleList()
        self.tuneChannels = nn.ModuleList()
        self.Upsampple = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        self.bn0 = nn.BatchNorm2d(resolutions[0])
        for idx in range(len(resolutions) - 1):
          self.ConvList.append(DownsampleFPN(resolutions[idx],
                                          resolutions[idx + 1],
                                          True))

          self.tuneChannels.append(torch.nn.Conv2d(resolutions[idx + 1],
                                                   self.target_channel,
                                                   kernel_size = 3,
                                                   stride = 1,
                                                   padding = 1))
          if idx != len(resolutions) - 2:
            self.Upsampple.append(Upsample(self.target_channel, True))

        self.convOut = torch.nn.Conv2d(self.target_channel,
                                      self.target_channel,
                                      kernel_size = 1,
                                      stride = 1,
                                      padding = 0)
        self.norm_seq = nn.ModuleList()
        for idx in range((len(self.resolutions) - 2) * 2 + 2):

          self.norm_seq.append(nn.BatchNorm2d(self.target_channel))




    def forward(self, x):
        h = self.bn0(x)
        FPN_list = []

        for idx in range(len(self.resolutions)):


          if idx == 0:
            h = self.pool(self.Lrelu(self.ConvList[idx](h)))
          else:
            h = self.Lrelu(self.ConvList[idx](temp))

          temp = h
          h = self.Lrelu(self.tuneChannels[idx](h))
          FPN_list.append(h)
        count = 0
        for idx in reversed(range(len(self.resolutions))):
          if idx == 0:
            hold = self.norm_seq[count](hold)
            count += 1
            hold = self.convOut(hold + self.norm_seq[count](FPN_list[idx]))
            break
          if idx == len(self.resolutions) - 1:
            hold = self.Upsampple[idx - 1](FPN_list[idx])
          else:
            hold = self.norm_seq[count](hold)
            count += 1

            hold = hold + self.norm_seq[count](FPN_list[idx])
            count = count + 1
            hold = self.Upsampple[idx - 1](hold)

        return hold


In [ ]:
class depth_phase1_block(nn.Module):
  def __init__(self, in_cha, out_cha):
    super().__init__()
    self.conv = DownsampleFPN(in_cha, out_cha, True)
    # self.Lrelu = nonlinearity
    self.bn = nn.BatchNorm2d(out_cha)
    self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
  def forward(self, depth):
    return self.Lrelu(self.bn(self.conv(depth)))


In [ ]:
class depth_phase2_block(nn.Module):
  def __init__(self, in_cha, out_cha):
    super().__init__()
    self.conv = torch.nn.Conv2d(in_cha,
                                out_cha,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1)
    # self.Lrelu = nonlinearity
    self.bn = nn.BatchNorm2d(out_cha)
    self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
  def forward(self, depth):
    return self.Lrelu(self.bn(self.conv(depth)))


In [ ]:
class depth_encode(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config.copy()
        resolution = config['data']['image_size']
        self.targeted_size = self.config['model']['depth_enc_targeted_size']
        self.enc_channels = self.config['model']['depth_enc_channels'].copy()
        # targeted_size = 64
        # enc_channels = [4, 16, 64, 256]
        enc_channels = self.enc_channels.copy()
        enc_channels.insert(0, 1)

        phase1 = 0
        while True:
          phase1 = phase1 + 1
          resolution = resolution / 2
          if resolution == self.targeted_size:
            break
        phase2 = len(self.enc_channels) - phase1
        self.phase1_model = nn.ModuleList()
        self.phase2_model = nn.ModuleList()

        for idx in range(phase1):
          self.phase1_model.append(depth_phase1_block(enc_channels[idx],
                                                      enc_channels[idx + 1]))
        for idx in range(phase2):
          self.phase2_model.append(depth_phase2_block(enc_channels[phase1 + idx],
                                                      enc_channels[phase1 + idx + 1]))

    def forward(self, depth):
        h = depth.unsqueeze(1)

        for idx in range(len(self.phase1_model)):
          h = self.phase1_model[idx](h)

        for idx in range(len(self.phase2_model)):
          h = self.phase2_model[idx](h)

        return h


In [ ]:
class depth_decode(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config.copy()
        self.ch = config['model']['ch']
        # self.ch = 128
        self.resolution = config['data']['image_size']
        # self.resolution = 256
        self.targeted_size = self.config['model']['depth_enc_targeted_size']
        # self.targeted_size = 64
        count = 0
        tmp = self.targeted_size
        while True:
          if tmp == self.resolution:
            break
          count = count + 1
          tmp = tmp * 2

        self.decode = nn.ModuleList()
        in_cha = self.ch
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        for idx in range(count):
          out_cha = in_cha / 4
          self.decode.append(UpsampleFPN(in_cha, out_cha, True))
          in_cha = out_cha
        self.final_conv = torch.nn.Conv2d(int(out_cha),  # this conv let the size unchanged
                                        1,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)








    def forward(self, pred):
        for idx in range(len(self.decode)):
          pred = self.decode[idx](pred)
        pred = self.final_conv(pred)
        pred = pred.squeeze(1)





        return pred

In [ ]:
from Model.functions.functions import get_beta_schedule, get_index_from_list, get_timestep_embedding
class DiffusionModel:
    def __init__(self, beta_schedule = 'linear', start_schedule=0.0001, end_schedule=0.02, timesteps = 300):
        self.start_schedule = start_schedule
        self.end_schedule = end_schedule
        self.timesteps = timesteps

        """
        if
            betas = [0.1, 0.2, 0.3, ...]
        then
            alphas = [0.9, 0.8, 0.7, ...]
            alphas_cumprod =      [0.9, 0.9 * 0.8, 0.9 * 0.8, * 0.7, ...]
            alphas_cumprod_prev = [1,   0.9, 0.9 * 0.8, 0.9 * 0.8 * 0.7]


        """
        betas = get_beta_schedule(beta_schedule, beta_start = start_schedule, beta_end = end_schedule, num_diffusion_timesteps = timesteps)
        self.betas = torch.tensor(betas)


        self.alphas = 1 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, axis=0).to(device)
        self.alphas_cumprod_prev = torch.cat(
            [torch.ones(1).to(device), self.alphas_cumprod[:-1]], dim=0
        )
    def forward(self, x_0, t, device):
        """
        x_0: (B, C, H, W)
        t: (B,)
        """


        noise = torch.randn_like(x_0)

        sqrt_alphas_cumprod_t = get_index_from_list(self.alphas_cumprod.sqrt(), t.to(torch.int64), x_0.shape)

        sqrt_one_minus_alphas_cumprod_t = get_index_from_list(torch.sqrt(1. - self.alphas_cumprod), t.to(torch.int64), x_0.shape)

        mean = sqrt_alphas_cumprod_t.to(device) * x_0.to(device)
        variance = sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device)

        return mean + variance, noise.to(device) # mean為x_0乘以alpha bar, variance 為 noise 乘以 1-alpha bar

    def backward(self, model, image, weight_path, skip, eta = 0.1):
        checkpoint = torch.load(weight_path, map_location = torch.device(device))
        model.load_state_dict(checkpoint['model_state_dict'])
        with torch.inference_mode():


            # seq = range(0, timesteps, skip)   # 這是原版
            # seq_next = [-1] + list(seq[:-1])

            seq = range(1, timesteps, skip)
            seq_next = [0] + list(seq[:-1])



            # seq =      [1, 2, 3]
            # seq_next = [0, 1, 2]
            x0_preds = []
            depth = torch.randn([image.shape[0], image.shape[-1], image.shape[-1]]).to(torch.float32)
            print(depth.dtype)
            xs = [depth]
            n = image.shape[0]



            for i, j in zip(reversed(seq), reversed(seq_next)):
                t = (torch.ones(n) * i).to(image.device)
                next_t = (torch.ones(n) * j).to(image.device)

                at = self.alphas_cumprod.gather(-1, t.to(torch.int64))

                at_next = self.alphas_cumprod.gather(-1, next_t.to(torch.int64))

                xt = xs[-1].to(device) # x_t
                x0_t = model(image, xt, t, sampling = True) # episolon t (predicted)

                x0_preds.append(x0_t.to(device))
                et = (-1 * x0_t * (at.sqrt()) - xt) / (at.sqrt())
                c1 = (
                    eta * ((1 - at / at_next) * (1 - at_next) / (1 - at)).sqrt() # c1是var in the distribution, which is at ddim page 5
                                                                                                # can make the sampling process of x_{t - 1} become identical
                                                                                                # as ddpm
                )
                c2 = ((1 - at_next) - c1 ** 2).sqrt()
                xt_next = at_next.sqrt() * x0_t + c1 * torch.randn_like(x0_t) + c2 * et # 這個xt_next也是x_(t-1) 取法是ddim 裡面定義的q(x_(t-1)|x_t, x_0)
                xs.append(xt_next.to(device))
            return xs, x0_preds



In [ ]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        ch, out_ch, ch_mult = config['model']['ch'], config['model']['out_ch'], tuple(config['model']['ch_mult'])
        num_res_blocks = config['model']['num_res_blocks']
        attn_resolutions = config['model']['attn_resolutions']
        dropout = config['model']['dropout']
        in_channels = config['model']['in_channels']
        resolution = config['data']['image_size']
        resamp_with_conv = config['model']['resamp_with_conv']
        num_timesteps = config['diffusion']['num_diffusion_timesteps']
        depth_enc_channels = config['model']['depth_enc_channels']
        if config['model']['type'] == 'bayesian':
            self.logvar = nn.Parameter(torch.zeros(num_timesteps))
        self.fpn = FPN(config)
        self.ch = ch

        self.temb_ch = self.ch*4
        self.num_resolutions = len(ch_mult)
        self.num_res_blocks = num_res_blocks
        self.resolution = resolution
        self.in_channels = in_channels

        # timestep embedding
        self.temb = nn.Module()
        self.temb.dense = nn.ModuleList([
            torch.nn.Linear(self.ch,
                            self.temb_ch),
            torch.nn.Linear(self.temb_ch,
                            self.temb_ch),
        ])

        '''
        # timestep embedding for diffusion---vvv
        self.temb.diff1 = nn.Linear(1, 1)
        self.temb.diff2 = nn.Linear(1, 1)
        # timestep embedding for diffusion---^^^
        '''


        self.depth_encode = depth_encode(config)

        # diffusion process ---vvv
        self.beta_schedule = config['diffusion']['beta_schedule']
        self.start_schedule = config['diffusion']['beta_start']
        self.end_schedule = config['diffusion']['beta_end']
        self.timesteps = config['diffusion']['num_diffusion_timesteps']
        self.diffusion_process = DiffusionModel(self.beta_schedule, self.start_schedule, self.end_schedule, self.timesteps)
        # diffusion process ---^^^



        # downsampling
        self.conv_in = torch.nn.Conv2d(depth_enc_channels[-1] * 2,
                                       self.ch,
                                       kernel_size=3,
                                       stride=1,
                                       padding=1)
# nonlinear

        curr_res = resolution
        in_ch_mult = (1,)+ch_mult
        self.down = nn.ModuleList()
        block_in = None
        for i_level in range(self.num_resolutions):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_in = ch*in_ch_mult[i_level]
            block_out = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks):

                block.append(ResnetBlock(in_channels=block_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            down = nn.Module()
            down.block = block
            down.attn = attn
            if i_level != self.num_resolutions-1:
                down.downsample = Downsample(block_in, resamp_with_conv)
                curr_res = curr_res // 2
            self.down.append(down)

        # middle
        self.mid = nn.Module()
        self.mid.block_1 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)
        self.mid.attn_1 = AttnBlock(block_in)
        self.mid.block_2 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)

        # upsampling
        self.up = nn.ModuleList()
        for i_level in reversed(range(self.num_resolutions)):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_out = ch*ch_mult[i_level]
            skip_in = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks+1):
                if i_block == self.num_res_blocks:
                    skip_in = ch*in_ch_mult[i_level] # 最後一個block時inchannel數可能變少
                block.append(ResnetBlock(in_channels=block_in+skip_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            up = nn.Module()
            up.block = block
            up.attn = attn
            if i_level != 0:
                up.upsample = Upsample(block_in, resamp_with_conv)
                curr_res = curr_res * 2
            self.up.insert(0, up)  # prepend to get consistent order, (0, up)代表在ModuleList()的0位置插入up這個Module

        # end
        self.norm_out = nn.BatchNorm2d(block_in)

        self.depth_decode = depth_decode(config)
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
    def forward(self, image, depth, t, sampling = False):
        # assert x.shape[2] == x.shape[3] == self.resolution # to check if the height and width are the same with the resolution

        # timestep embedding
        temb = get_timestep_embedding(t, self.ch).to(device)
        temb = self.temb.dense[0](temb)
        temb = self.Lrelu(temb)
        temb = self.temb.dense[1](temb)



        img_enc = self.fpn(image)
        depth = depth.to(torch.float32)
        # if sampling == False:
        depth = self.depth_encode(depth)


        # depth = depth.unsqueeze(1)
        # return img_enc, depth

        # diffusion process ---vvv
        if sampling == False:
            noisy_map, noise = self.diffusion_process.forward(depth, t, device = t.device)
            noisy_map = noisy_map.to(torch.float32)
            noise = noise.to(torch.float32)
        else:
            noisy_map = depth



        # diffusion process ---^^^


        # concat img_enc and noisy_map
        backbone_input = torch.cat([noisy_map, img_enc], dim = 1)
        # return backbone_input



        # downsampling down 裡面有好幾個元素，每個元素包含block(resblock), attn, downsample，其中attn只有幾個元素會有，downsample除了最後一個元素以外都有
        # 整個流程就是把x送進conv2D 然後送進down裡面經過resblock和部份attn downsample(conv2D)
        # hs = [self.conv_in(image)]

        hs = [self.conv_in(backbone_input)]

        for i_level in range(self.num_resolutions):
            for i_block in range(self.num_res_blocks):

                h = self.down[i_level].block[i_block](hs[-1], temb) # h 如果可以是attn就是attn 不然就是res, note that h是把值喂進去模塊後的值，要把hs的最後跟time embedded送進去
                if len(self.down[i_level].attn) > 0:
                    h = self.down[i_level].attn[i_block](h)
                hs.append(h)
            if i_level != self.num_resolutions-1:
                hs.append(self.down[i_level].downsample(hs[-1]))

        # middle
        h = hs[-1]
        h = self.mid.block_1(h, temb)
        h = self.mid.attn_1(h)
        h = self.mid.block_2(h, temb)

        # upsampling
        for i_level in reversed(range(self.num_resolutions)):
            for i_block in range(self.num_res_blocks+1):
                h = self.up[i_level].block[i_block](torch.cat([h, hs.pop()], dim=1), temb) # u-net的cat down
                if len(self.up[i_level].attn) > 0:
                    h = self.up[i_level].attn[i_block](h)
            if i_level != 0:
                h = self.up[i_level].upsample(h)

        # end
        h = self.norm_out(h)
        h = self.Lrelu(h)
        h = self.depth_decode(h)
        return h
